<font size="7">Tanner Sundwall- Decision Trees CS472</font>

In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn import tree
from sklearn.tree import export_text
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
import graphviz
from sklearn import metrics
import copy

<font size="5">ID3 Algorithm, Cross Validation Algorithm, etc.</font>

In [47]:
class TreeNode():

    def __init__(self, label, nodeType, feature):

        self.root = self
        self.children = []
        self.label = label
        self.type = nodeType
        self.feature = feature #which feature is this label associated with?


    def peek_children(self,mute=False):

        list = []
        if not mute:
            print("splits on:" + self.children[0].feature)
        for child in self.children:
            list.append(child.label)

        return list


class DTClassifier():

    def get_entropy(self, x): #ret float

        tot = 0

        for featureValue in list(set(x)):

            n_v = x.count(featureValue) / len(x)

            sub = -n_v * np.log2(n_v)
            tot += sub

        return tot

    def select_feature(self, df, remainingFeatures, outcomeLabel, mute=False): #x is df-like structure

        maxInfoGainFeature = ""
        maxInfoGain = 0
        E_c = self.get_entropy(list(df[outcomeLabel]))

        for feature in list(df.columns):
            if feature == outcomeLabel:
                continue
            elif feature not in remainingFeatures:
                continue
            tot = 0

            for featureValue in list(set(df[feature])):

                subDf = df[df[feature]==featureValue]
                E_f = self.get_entropy(list(subDf[outcomeLabel]))
                E_f = (len(subDf)/len(df)) * E_f
                tot += E_f

            if (E_c - tot) > maxInfoGain:
                maxInfoGain = E_c - tot
                maxInfoGainFeature = feature

        if maxInfoGain == 0: #trivial in this case
            return remainingFeatures[0]

        if not mute:
            print(maxInfoGain)

        return maxInfoGainFeature

    def get_majority_outcome(self, df, outcomeLabel):

        maxCount = 0
        maxCountOutcome = ""

        for outcomeValue in list(set(df[self.outcomeLabel])):

            currCount = list(df[self.outcomeLabel]).count(outcomeValue)

            if currCount > maxCount:
                maxCount = currCount
                maxCountOutcome = outcomeValue

        return maxCountOutcome

    def create_tree(self, df, remainingFeatures, rootName, currFeature, outcomeLabel, mute):

        rootNode = TreeNode(rootName, "label", currFeature)

        if len(list(set(df[outcomeLabel]))) == 1:
            child = TreeNode(list(set(df[outcomeLabel]))[0], "resolution", "resolution")
            child.root = rootNode
            rootNode.children.append(child)

            return rootNode

        elif len(remainingFeatures) == 1:

            child = TreeNode(self.get_majority_outcome(df, outcomeLabel), "resolution", "resolution")
            child.root = rootNode
            rootNode.children.append(child)

            return rootNode

        else:
            F_hat = self.select_feature(df, copy.deepcopy(remainingFeatures), outcomeLabel, mute)

            remainingFeatures.remove(F_hat)
            for featureValue in list(set(df[F_hat])):
                childBranch = self.create_tree(df[df[F_hat]==featureValue], copy.deepcopy(remainingFeatures), rootName=featureValue, currFeature = F_hat, outcomeLabel=outcomeLabel, mute=mute)
                childBranch.root = rootNode
                rootNode.children.append(childBranch)

            return rootNode

    def predict(self, df, treeRoot, outcomeLabel):

        resolutions = []
        allOutcomes = list(df[outcomeLabel])

        for obs in df.itertuples():

            noResolution = True
            currNode = treeRoot
            currFeature = treeRoot.children[0].feature

            while noResolution:

                value = getattr(obs,currFeature)
                childrenList = currNode.peek_children(mute=True)

                try:
                    branchIndex = childrenList.index(value)

                except:
                    defaultValue = max(set(allOutcomes), key=allOutcomes.count)
                    resolutions.append(defaultValue)
                    break

                currNode = currNode.children[branchIndex]

                if currNode.children[0].feature == "resolution":
                    resolutions.append(currNode.children[0].label)
                    noResolution = False

                else:
                    currFeature = currNode.children[0].feature

        return resolutions

    def score(self, vals, preds):

        compareDf = pd.DataFrame([vals, preds]).transpose()
        compareDf['result'] = compareDf[0] == compareDf[1]

        return np.mean(list(compareDf['result']))

def crossValidate(arffString, k, outcomeLabel, convertColNames=False, mute=True):

    data = arff.loadarff(arffString)

    df = pd.DataFrame(data[0])

    if convertColNames:
        convertDashColNames(df)

    dtObj = DTClassifier()
    df = df.sample(frac=1)
    resultsTbl = []
    testSetSize = int(len(df) / k)

    for testSet in range(1, k+1):

        testSetIndexStart = (testSet - 1) * testSetSize
        testSetIndexEnd = testSet * testSetSize
        testSetDf = df[testSetIndexStart:testSetIndexEnd]
        trainingSetDf = pd.concat([df[:testSetIndexStart],df[testSetIndexEnd:]])
        tree = dtObj.create_tree(trainingSetDf, copy.deepcopy(list(trainingSetDf.columns)), rootName="start", currFeature = "na", outcomeLabel=outcomeLabel, mute=mute)

        trainingPreds = dtObj.predict(trainingSetDf, tree, outcomeLabel)
        trainingAccuracy = dtObj.score(list(trainingSetDf[outcomeLabel]), trainingPreds)

        testPreds = dtObj.predict(testSetDf, tree, outcomeLabel)
        testAccuracy = dtObj.score(list(testSetDf[outcomeLabel]), testPreds)

        resultsTbl.append([trainingAccuracy,testAccuracy])

    return resultsTbl

def convertDashColNames(df):

    listCols = []
    for each in list(df.columns):
        listCols.append(each.replace("-", "_"))

    df.columns = listCols

def trainModel(arffString, outcomeLabel, mute):

    data = arff.loadarff(arffString)
    df = pd.DataFrame(data[0])

    dtObj = DTClassifier()
    return dtObj.create_tree(df, copy.deepcopy(list(df.columns)), rootName="start", currFeature="na", outcomeLabel=outcomeLabel, mute=mute)

def scoreModel(modelTree, arffString, outcomeLabel):

    data = arff.loadarff(arffString)
    df = pd.DataFrame(data[0])

    dtObj = DTClassifier()
    preds = dtObj.predict(df, modelTree, outcomeLabel)

    return dtObj.score(list(df[outcomeLabel]), preds)

<font size="5">1.1- Debug (Lenses Dataset)</font>

In [48]:
print("Info Gain Splits:\n")
modelTree = trainModel("lenses.arff", "contact_lenses", False)
print("\nMean Test Set Accuracy:\n")
scoreModel(modelTree, "lenses_test.arff", "contact_lenses")

Info Gain Splits:

0.5487949406953985
0.7704260414863778
0.4591479170272448
0.9182958340544896
0.3166890883150208
1.0

Mean Test Set Accuracy:



0.3333333333333333

<font size="5">1.2- Evaluation (Zoo Dataset)</font>

In [49]:
print("Info Gain Splits:\n")
modelTree = trainModel("zoo.arff", "type", False)
print("\nMean Test Set Accuracy:\n")
scoreModel(modelTree, "zoo_test.arff", "type")

Info Gain Splits:

1.3630469031539394
0.6892019851173654
0.8631205685666308
0.7219280948873623
0.7219280948873623
0.8256265261578954
0.8865408928220899
0.6962122601251458
0.9852281360342515

Mean Test Set Accuracy:



0.147

<font size="5">2.2- Cross Validation (Cars Dataset)</font>

In [50]:
report = crossValidate("cars.arff", 10, "class")
reportDf = pd.DataFrame(report, columns=["training","test"])
print(reportDf)
print("\nmean test accuracy:\n")
np.mean(reportDf["test"])

   training      test
0       1.0  0.877907
1       1.0  0.901163
2       1.0  0.883721
3       1.0  0.843023
4       1.0  0.930233
5       1.0  0.901163
6       1.0  0.877907
7       1.0  0.872093
8       1.0  0.906977
9       1.0  0.906977

mean test accuracy:



0.8901162790697675

<font size="5">2.3- Cross Validation (Voting Dataset)</font>

In [51]:
report = crossValidate("voting_with_missing.arff", 10, "Class", convertColNames=True)
reportDf = pd.DataFrame(report, columns=["training","test"])
print(reportDf)
print("\nmean test accuracy:\n")
np.mean(reportDf["test"])

   training      test
0       1.0  0.953488
1       1.0  0.930233
2       1.0  0.906977
3       1.0  1.000000
4       1.0  0.883721
5       1.0  0.883721
6       1.0  0.930233
7       1.0  0.953488
8       1.0  0.976744
9       1.0  0.953488

mean test accuracy:



0.9372093023255814

<font size="5">2.4- Discuss Your Results</font>

**Results are non-deterministic due to row shuffling; printed results may vary slightly from report**

Both cross-validation results exhibited high accuracy. The mean accuracy across the 10 test folds for “Cars” was 94.88%. No fold did any worse than 90.7%, and no better than 98%. All training sets saw 100% accuracy.

For the voting dataset, the training data saw 93.48% accuracy, only slightly lower than for “Cars”. No fold did any worse than 88.37%, and no better than 97.7%. Again, all training sets saw 100% accuracy.

We would expect to see lower accuracies if the training set was smaller in proportion to the test set (like in the ‘Lenses’ case). In such cases, sample sizes may not be high enough to best approximate the best attributes to place shallower in the tree.

Why were the training sets so accurate? Because the tree was induced from this data, we expect very high accuracy. 100% accuracy occurs when all permutations of data only produce one outcome (that is, observation with all the same attributes have homogeneous results). There are no cases in which all features have been used, and a mixed set of results exist when deciding on the leaf node. In cases where training sets are much larger (or noise is much greater), we would expect less than 100% accuracy when testing the training set.



<font size="5">3.1- 'Cars' Decision Tree Analysis</font>

In [52]:
modelTree = trainModel("cars.arff", "class",True)
print(modelTree.peek_children())
print("\nb'high':")
print(modelTree.children[2].peek_children())
print("\nb'med':")
print(modelTree.children[0].peek_children())
print("\nb'low':")
print(modelTree.children[1].peek_children())

splits on:safety
[b'med', b'high', b'low']

b'high':
splits on:resolution
[b'unacc']

b'med':
splits on:persons
[b'4', b'more', b'2']

b'low':
splits on:persons
[b'4', b'more', b'2']


The first branch in the “Cars” decision tree was split on the feature “safety”. This means that, across all features in the dataset, ‘safety’ had the highest information gain. This means that its sub-branches had low entropy, meaning that the splits were relatively homogenous compared to other features.

If we dig deeper into the “high safety” branch, the “persons” attribute was the next to be split. This means that among the subset of data with high safety, the number of persons gives the maximum information gain for the next split. If we sift down the “mid safety” branch, we get the same split.

What’s interesting is that if we traverse the “low safety” branch, we immediately arrive at a homogeneous solution; all cases of low safety lead to class “unacc”. Not a single datapoint with this attribute had class “acc”, and we know that for each cross validation tree that was induced, a similar result will follow (“low safety”->”unacc”). The homogeneity of this branch is likely what propelled “safety” to being the first attribute chosen. 


<font size="5">3.2- 'Voting' Decision Tree Analysis</font>

In [53]:
modelTree = trainModel("voting_with_missing.arff", "Class",True)
print(modelTree.peek_children())
print("\nb'y':")
print(modelTree.children[0].peek_children())
print("\nb'n':")
print(modelTree.children[1].peek_children())
print("\nb'?':")
print(modelTree.children[2].peek_children())
print("\nb'y',b'y':")
print(modelTree.children[0].children[0].peek_children())
print("\nb'y',b'n':")
print(modelTree.children[0].children[1].peek_children())
print("\nb'y',b'?':")
print(modelTree.children[0].children[2].peek_children())

splits on:physician-fee-freeze
[b'?', b'y', b'n']

b'y':
splits on:mx-missile
[b'?', b'n', b'y']

b'n':
splits on:synfuels-corporation-cutback
[b'?', b'n', b'y']

b'?':
splits on:adoption-of-the-budget-resolution
[b'?', b'y', b'n']

b'y',b'y':
splits on:resolution
[b'republican']

b'y',b'n':
splits on:resolution
[b'democrat']

b'y',b'?':
splits on:anti-satellite-test-ban
[b'?', b'y', b'n']


In the “Voting” decision tree, the first branch is on “physician_fee_freeze”. As I will note later, unknown attributes were kept as their own intrinsic values. By being split on first, we know that this attribute holds lots of information about the class of the individual (Democrat/Republican). 

Down the “yes” branch, “synfuels_corporation_cutback” is the next branching attribute. This is a different attribute than the branching attribute for the “no” and “?” branches (“adoption_of_the_budget_resolution” and “mx_missle” respectively). This means that across all three subsets of data, a different attribute contains the most amount of remaining information. 

If we dig a bit deeper into those who said yes to 'physician-fee-freeze', their response to 'synfuels-corporation-cutback' leads to a variety of next splits. It is quite apparent that the split feature for any given attribute value seems to differ greatly. We also see that a resolution (leaf node) is reached for those who had an unknown response to the 'synfuels-corporation-cutback' question; although this may be a case of overfitting, where only one or a few individuals exist in this leaf node, making it easily homogenous.

<font size="5">3.3- Handling Unknown Attributes</font>

Given the large number of unknown attributes in the “Voting” dataset, I made the assumption that “?” was its own attribute; that it holds some intrinsic value that can predict class (Democrat/Republican). Essentially, I made the assumption that unknown values are not randomly distributed across outcomes. Without exact knowledge of the data, we can also potentially infer that an unknown response may mean “I do not know”, which may result to more Republicans or Democrats. 

<font size="5">4.1- Voting Dataset- Scikit</font>

With the DecisionTreeClassifier() method in scikit-learn, we run into some limitations; inputs must be numerical. This also means that splits can only be binary, and therefore we can’t have a feature with any more than two potential values. In these cases, a feature encoded as (0,1,2) will be treated as ordinal, which isn’t usually a good assumption. Because of this complication, features with more than two attributes must be one-hot encoded, which may lead to a very deep tree. With numerical data, it is also possible to split on a feature more than once, which may not make sense in many cases. 

In the case of the voting dataset, we only have features with binary classifiers, so we shouldn't run into any issues.

In [54]:
data = arff.loadarff("voting_with_missing.arff")
df = pd.DataFrame(data[0])

encode = preprocessing.LabelEncoder() #label encoder fine for binary variables; no need to hot encode
df = df.apply(encode.fit_transform)

y = pd.DataFrame(df['Class'])
del df[df.columns[-1]]

dfTrain = df[:int(len(df)/2)]
yTrain = y[:int(len(df)/2)]
dfTest = df[int(len(df)/2):]
yTest = y[int(len(df)/2):]

cl = DecisionTreeClassifier(max_depth=3)
cl = cl.fit(dfTrain,yTrain)

preds = cl.predict(dfTest)
print("Test Accuracy:")
metrics.accuracy_score(preds, yTest)

Test Accuracy:


0.9220183486238532

Data was split in half for the training and test data. I wanted to see if a simpler tree could perform similarly to a default configuration, so i set max_depth=4. The default model and the modified model performed very similarly, so I kept the simpler one (both had accuracy ~92%). 

This test accuracy performs very similarly to the one cross validated on my ID3 algorithm, which saw 93% test accuracy. It would be interesting to see how much better the results would be with this model if I were to implement cross validation methods.

Looking at the tree below, it is quite a bit simpler than my model which didn't limit tree depth. This one is much easier to read, and may be more interpretable across domains. Another reason for its simplicity is the strict binary splits implemented. 'physician-fee-freeze' is still the first attribute to split on, but the branch structure deviates from mine greatly after that. 

In [55]:
print(export_text(cl, feature_names=list(df.columns)))

|--- physician-fee-freeze <= 1.50
|   |--- aid-to-nicaraguan-contras <= 0.50
|   |   |--- crime <= 1.50
|   |   |   |--- class: 0
|   |   |--- crime >  1.50
|   |   |   |--- class: 1
|   |--- aid-to-nicaraguan-contras >  0.50
|   |   |--- class: 0
|--- physician-fee-freeze >  1.50
|   |--- duty-free-exports <= 1.50
|   |   |--- synfuels-corporation-cutback <= 1.50
|   |   |   |--- class: 1
|   |   |--- synfuels-corporation-cutback >  1.50
|   |   |   |--- class: 1
|   |--- duty-free-exports >  1.50
|   |   |--- anti-satellite-test-ban <= 1.50
|   |   |   |--- class: 0
|   |   |--- anti-satellite-test-ban >  1.50
|   |   |   |--- class: 1



<font size="5">4.2- Example Dataset- Predicting NFL wins and losses (Scikit)</font>

In [56]:
nflDf = pd.read_csv("nflEdited.csv")
y = pd.DataFrame(nflDf['Result'])
del nflDf[nflDf.columns[-1]]

nflDfTrain = nflDf[:500]
yTrain = y[:500]
nflDfTest = nflDf[500:]
yTest = y[500:]

cl = DecisionTreeClassifier(max_depth=4,criterion="entropy")
cl = cl.fit(nflDfTrain,yTrain)

nflPreds = cl.predict(nflDfTest)
print("Test Accuracy:")
metrics.accuracy_score(nflPreds, yTest)

Test Accuracy:


0.6895161290322581

I exported data of the last 1000 NFL games from Stathead to complete this analysis. I used half the dataset as training data and half as test data. The goal is to predict whether the home team won or lost the game, given several team/player stats from that game.

The NFL dataset saw 65% accuracy with all parameters untouched. Using just entropy, it saw 67% accuracy. To reduce dimensionality and potential overfitting, I tried an iteration where I set the max depth to 3. It yielded an accuracy of 70%, which may be worth reducing dimensionality for, especially if it needs to be interpreted visually. No other parameter tweaks got the accuracy over 70%. The accuracy above reflects a model that uses entropy and has max depth of 4.


<font size="5">5- Visualizing NFL Decision Tree</font>

As seen in the tree below, passer rating was the first feature to be split, which follows with general football intuition. When passer rating was below 111.3, number of turnovers forced was then evaluated. At any given level, the children nodes seem to evaulate different features, making it a bit hard to follow. Other common feature splits among the tree include DY/P (defensive yards allowed per play), penalty yards, and total offense allowed. 

Another reason I wanted to limit the depth of the tree was to see which features were chosen by the algorithm. The ones in the shallow portion and the ones i listed above are all features that I had hypothesized as having predictive value, and surprisingly correlate with talking points/discussion by sports analysts and commentators.

At the leaf nodes, the visualization allows us to see the homogeneity among outcomes. One that stands out is the furthest left leaf node; all 154 observations at this branch were Losses. The intution along this branch checks out; low passer rating, low turnovers forced, low offensive yards per attempt, and low opposing penalty yards.

While it is a fairly simply tree to glaze over, some of the intution seems to fall apart at lower levels though, especially when a feature is split on a second time. When passer rating is over 111.3, it immediately splits again. Overall, I think given the numerical nature of the data and the poor accuracy (68% seems low), there are likely other algorithms that would be much more effective. 

In [58]:
!jupyter nbconvert --execute --to html report.ipynb

[NbConvertApp] Converting notebook report.ipynb to html
[NbConvertApp] Writing 654646 bytes to report.html
